# Introduction
State notebook purpose here

### Imports
Import libraries and write settings here.

In [6]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython, display
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sn

# Import APTs
from assets.api_keys import api_key

# Additional modules
from citipy import citipy
import requests
from requests.auth import HTTPBasicAuth
import json

# Analysis/Modeling
Do work here

### Create a uniformly distributed DataFrame of cities across Latitudes and Longitudes 

In [3]:
# Create Long and Latitude file if it hasn't already been created
try:
    city_df = pd.read_pickle('assets/city_df')
except IOError:
    # Get a representative sample of longitude and latitudes
    lats = np.random.uniform(-90., 90., 1500)
    longs = np.random.uniform(-180., 180., 1500)
    city_df = pd.DataFrame({'Latitude': lats, 'Longitude': longs })
    pd.to_pickle(city_df, 'assets/city_df')

In [4]:
city_df.head()

,Latitude,Longitude
0,66.923445,-178.524447
1,-28.691295,105.165906
2,-58.509280,111.378456
3,73.326901,-111.369206
4,13.079430,-106.503989


In [7]:
# Create the base url
base_url = f'http://api.openweathermap.org/data/2.5/find?'

In [6]:
# Test the API
test_city = "London,uk"
params = {'q': city, 'APPID': api_key}
response = requests.get(base_url, params=params)
print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "cod": "200",
    "count": 1,
    "list": [
        {
            "clouds": {
                "all": 40
            },
            "coord": {
                "lat": 51.5085,
                "lon": -0.1258
            },
            "dt": 1553363746,
            "id": 2643743,
            "main": {
                "humidity": 71,
                "pressure": 1032,
                "temp": 283.32,
                "temp_max": 285.37,
                "temp_min": 280.93
            },
            "name": "London",
            "rain": null,
            "snow": null,
            "sys": {
                "country": "GB"
            },
            "weather": [
                {
                    "description": "scattered clouds",
                    "icon": "03d",
                    "id": 802,
                    "main": "Clouds"
                }
            ],
            "wind": {
                "deg": 80,
                "speed": 5.1
            }
        }
    ],
    "message": "ac

In [7]:
# Find cities closest to the urls (assuming they exist, otherwise resturn Nan)
def get_nearest_city(coor):
    city = citipy.nearest_city(coor['Latitude'], coor['Longitude'])
    return city.city_name + ',' + city.country_code
    
city_df['city'] = city_df.apply(get_nearest_city, axis=1)
city_df.head()

,Latitude,Longitude,city
0,66.923445,-178.524447,"egvekinot,ru"
1,-28.691295,105.165906,"carnarvon,au"
2,-58.509280,111.378456,"albany,au"
3,73.326901,-111.369206,"yellowknife,ca"
4,13.079430,-106.503989,"coahuayana,mx"


In [11]:
# Check for Nan cities
num_of_na = len(city_df[city_df['city'].isna()])
print(f'There were {num_of_na} cities')

There were 0 cities


# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here